# Classification

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Tutorial 3

$\newcommand{\trace}[1]{\operatorname{tr}\left\{#1\right\}}$
$\newcommand{\Norm}[1]{\lVert#1\rVert}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\inner}[2]{\langle #1, #2 \rangle}$
$\newcommand{\DD}{\mathscr{D}}$
$\newcommand{\grad}[1]{\operatorname{grad}#1}$
$\DeclareMathOperator*{\argmin}{arg\,min}$

Setting up the environment

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt

%matplotlib inline

## The data set

We will predict the incidence of diabetes based on various measurements (see [description](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes)). Instead of directly using the raw data, we use a normalised version, where the label to be predicted (the incidence of diabetes) is in the first column. Download the data from [the course website](https://machlearn.gitlab.io/isml2017/tutorial/diabetes_scale.csv).

Read in the data using ```np.loadtxt```.

In [5]:
# Solution goes here
data = np.loadtxt("diabetes_scale.csv",delimiter = ",")

## Classification via Logistic Regression

Implement binary classification using logistic regression for a data set with two classes. Make sure you use appropriate [python style](https://www.python.org/dev/peps/pep-0008/) and [docstrings](https://www.python.org/dev/peps/pep-0257/).

Use ```scipy.optimize.fmin_bfgs``` to optimise your cost function. ```fmin_bfgs``` requires the cost function to be optimised, and the gradient of this cost function. Implement these two functions as ```cost``` and ```grad``` by following the equations in the lectures.

Implement the function ```train``` that takes a matrix of examples, and a vector of labels, and returns the maximum likelihood weight vector for logistic regresssion. Also implement a function ```test``` that takes this maximum likelihood weight vector and the a matrix of examples, and returns the predictions. See the section **Putting everything together** below for expected usage.

We add an extra column of ones to represent the constant basis.

In [6]:
data = np.hstack([data, np.ones((data.shape[0], 1))]) # add a column of ones
data[:5,:]

array([[-1.        , -0.294118  ,  0.487437  ,  0.180328  , -0.292929  ,
        -1.        ,  0.00149028, -0.53117   , -0.0333333 ,  1.        ],
       [ 1.        , -0.882353  , -0.145729  ,  0.0819672 , -0.414141  ,
        -1.        , -0.207153  , -0.766866  , -0.666667  ,  1.        ],
       [-1.        , -0.0588235 ,  0.839196  ,  0.0491803 , -1.        ,
        -1.        , -0.305514  , -0.492741  , -0.633333  ,  1.        ],
       [ 1.        , -0.882353  , -0.105528  ,  0.0819672 , -0.535354  ,
        -0.777778  , -0.162444  , -0.923997  , -1.        ,  1.        ],
       [-1.        , -1.        ,  0.376884  , -0.344262  , -0.292929  ,
        -0.602837  ,  0.28465   ,  0.887276  , -0.6       ,  1.        ]])

In [1]:
# Solution goes here
# scipy.optimize.fmin_bfgs
# fmin_bfgs
def sigmoid(x):
    return(1/(1+np.exp(-x)))

def phi(x):
    return(x)

def predict(w,x):
    return(sigmoid(phi(x).dot(w)))

def cost(w,x,t):
    return(-(t.T.dot(np.log(predict(w,x))) + (1-t).T.dot(np.log(predict(w,-x)))))

def grad(w,x,t):
    return((predict(w,x)-t).T.dot(phi(x)))


def train(x,t):
    w = np.ones(x.shape[1])*1e-8
    w = opt.fmin_bfgs(cost, x0 = w, fprime = grad, args = (x,t))
    return(w)
    

def test(w,x,t):
    print(cost(w,x,t))
    return(predict(x,w))

# Test
# x = data[:,1:]
# t = data[:,0]
# t = t>0

# n = len(x)
# train_x = x[:n/2,:]
# train_y = t[:n/2]
# test_x = x[n/2:,:]
# test_y = t[n/2:]

# w = train(train_x,train_y)
# pred_y = test(w,test_x,test_y)

## Performance measure

There are many ways to compute the [performance of a binary classifier](http://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers). The key concept is the idea of a confusion matrix or contingency table:

|              |    | Label |    |
|:-------------|:--:|:-----:|:--:|
|              |    |  +1   | -1 |
|**Prediction**| +1 |    TP | FP |
|              | -1 |    FN | TN |

where
* TP - true positive
* FP - false positive
* FN - false negative
* TN - true negative

Implement three functions, the first one which returns the confusion matrix for comparing two lists (one set of predictions, and one set of labels). Then implement two functions that take the confusion matrix as input and returns the **accuracy** and **balanced accuracy** respectively. Accuracy is defined as the number of correct classifications divided by the total number of examples. The balanced accuracy is the average accuracy of each class, that is the accuracy when the true class is positive and the accuracy when the true class is negative (averaged).


In [20]:
# Solution goes here

def confusion_matrix(pred,truth):
    N = len(pred)
    TP_array =  [ pred[i] for i in range(N) if pred[i] == 1 and truth[i] == 1]
    FP_array =  [ pred[i] for i in range(N) if pred[i] == 1 and truth[i] == -1]
    TN_array =  [ pred[i] for i in range(N) if pred[i] == -1 and truth[i] == -1]
    FN_array =  [ pred[i] for i in range(N) if pred[i] == -1 and truth[i] == 1]
    
    res = {"TP":len(TP_array),"FP":len(FP_array),"TN":len(TN_array),"FN":len(FN_array) }
    return(res)

def accuracy(confusion_matrix):
    
    N = sum(confusion_matrix.values())
    accuracy = (confusion_matrix["TP"] + confusion_matrix["TN"])/N
    return(accuracy)

def balanced_accuracy(confusion_matrix):
    BA1 = confusion_matrix["TP"]/(confusion_matrix["TP"] + confusion_matrix["FN"])
    BA2 = confusion_matrix["TN"]/(confusion_matrix["FP"] + confusion_matrix["TN"])
    return((BA1 + BA2)/2)

## Putting everything together

Consider the following code, which trains on all the examples, and predicts on the training set. Discuss the results.

In [21]:
y = data[:,0]
X = data[:,1:]
theta_best = train(X, y)
pred = predict(theta_best, X)
cmatrix = confusion_matrix(pred, y)
[accuracy(cmatrix), balanced_accuracy(cmatrix)]

         Current function value: nan
         Iterations: 8
         Function evaluations: 119
         Gradient evaluations: 119
0.0


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:5: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:5: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:5: RuntimeWarning: overflow encountered in exp


[0.9107692307692308, 0.5]

### Solution description

## Fisher's discriminant

In the lectures, you saw that the Fisher criterion
$$
J(w) = \frac{w^T S_B w}{w^T S_W w}
$$
is maximum for Fisher's linear discriminant.

Define $S_B$ and $S_W$ as in the lectures and prove this result.

### Solution description

### Solution description

## (optional) Effect of regularization parameter

By splitting the data into two halves, train on one half and report performance on the second half. By repeating this experiment for different values of the regularization parameter $\lambda$ we can get a feeling about the variability in the performance of the classifier due to regularization. Plot the values of accuracy and balanced accuracy for at least 3 different choices of $\lambda$. Note that you may have to update your implementation of logistic regression to include the regularisation parameter.


In [ ]:
### Solution

def split_data(data):
    """Randomly split data into two equal groups"""
    np.random.seed(1)
    N = len(data)
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:int(N/2)]
    test_idx = idx[int(N/2):]

    X_train = data[train_idx, 1:]
    t_train = data[train_idx, 0]
    X_test = data[test_idx, 1:]
    t_test = data[test_idx, 0]
    
    return X_train, t_train, X_test, t_test

X_train, t_train, X_test, t_test = split_data(data)


### Solution description